In [1]:
import pandas as pd
import numpy as np

In [60]:
data = pd.read_csv("/content/train.csv", engine = 'python', encoding='utf-8', on_bad_lines = 'skip')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [61]:
data.shape

(20822, 5)

In [62]:
data.isna().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

Since this data has null value, and we cannot replace text data with any values. So we have to drop missing values.

In [63]:
data = data.dropna()
data.shape

(18285, 5)

In [118]:
data = data.loc[(data['label'] == '1') | (data['label'] == '0')]

In [119]:
X = data.drop(labels = 'label', axis = 1)
y = data['label']
X.shape, y.shape

((18283, 4), (18283,))

In [120]:
import tensorflow
print(tensorflow.__version__)

2.15.0


In [121]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [122]:
 vocab_size = 5000

## **One Hot Representation**

In [123]:
message = X.copy()

In [124]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [125]:
message = message.reset_index(drop=True)
message = message.drop(labels = 'id', axis = 1)

In [126]:
message.head(10)

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [127]:
import nltk
from nltk.corpus import stopwords
import re

from nltk.stem.porter import PorterStemmer
stem = PorterStemmer()

In [128]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
corpus = []

for sentence in range(len(message)):
    review = re.sub('[^a-zA-Z0-9]', ' ', message['title'][sentence]).lower().split()
    review = [stem.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [130]:
one_hot_repr = [one_hot(word, vocab_size) for word in corpus]
print(one_hot_repr)

[[3341, 3457, 4222, 4552, 3395, 2463, 2407, 4277, 4767, 4858], [4536, 3923, 4884, 1515, 114, 4568, 979], [3996, 900, 1190, 3420], [1965, 2936, 4654, 2662, 4261, 227, 1092], [513, 114, 1878, 829, 2738, 2249, 114, 1409, 4339, 782], [2136, 2011, 2799, 1122, 1666, 3847, 2705, 2019, 4677, 1230, 1325, 1009, 1299, 3086, 979], [230, 2912, 4176, 368, 2810, 4821, 784, 2793, 1284, 1903, 2486], [2876, 2405, 624, 2958, 4547, 1419, 3847, 2159, 1284, 1903, 2486], [4200, 4598, 3492, 1218, 1061, 68, 2769, 2063, 3847, 2635], [448, 85, 4239, 1625, 2174, 3329, 3383, 2275], [1554, 486, 822, 3067, 4669, 1820, 2356, 2995, 281, 6, 1575], [4261, 2687, 3395, 68, 3847, 4547], [3620, 1963, 4976, 1593, 1030, 3281, 4301, 712, 2570], [2407, 3599, 4383, 3548, 4538, 2740, 4591, 1284, 1903, 2486], [3158, 1736, 2762, 1795, 171, 1284, 1903, 2486], [734, 241, 114, 538, 3990, 1932, 4411, 1831, 1060, 1251, 1552, 4693], [1818, 3969, 3923], [4979, 4224, 504, 4205, 3847, 3212, 4319, 979], [1429, 3382, 4884, 655, 2150, 3671, 35

## **Embedding Representation**

In [131]:
max_sentence_len = max([len(sentence.split()) for sentence in corpus])
print(max_sentence_len)
embedded_sentences = pad_sequences(one_hot_repr, padding='pre', maxlen=max_sentence_len)
print(embedded_sentences)

47
[[   0    0    0 ... 4277 4767 4858]
 [   0    0    0 ...  114 4568  979]
 [   0    0    0 ...  900 1190 3420]
 ...
 [   0    0    0 ... 1284 1903 2486]
 [   0    0    0 ... 3831 1809 1618]
 [   0    0    0 ... 2772 4543 3253]]


In [132]:
embedded_sentences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3341, 3457, 4222, 4552, 3395, 2463, 2407,
       4277, 4767, 4858], dtype=int32)

## **Model Trainer**

In [133]:
feature_dim = 50
model = Sequential()
model.add(Embedding(vocab_size, feature_dim, input_length = max_sentence_len))
model.add(Dropout(0.3))
model.add(LSTM(100)) # LSTM with 100 neurons
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 47, 50)            250000    
                                                                 
 dropout_4 (Dropout)         (None, 47, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [134]:
len(embedded_sentences), y.shape

(18283, (18283,))

In [135]:
X_final = np.array(embedded_sentences)
y_final = np.array(y)
X_final.shape, y_final.shape

((18283, 47), (18283,))

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 5378)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12798, 47), (5485, 47), (12798,), (5485,))

In [141]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [142]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size=70)

Epoch 1/10
183/183 [==============================] - 15s 68ms/step - loss: 0.3422 - accuracy: 0.8348 - val_loss: 0.1907 - val_accuracy: 0.9150
Epoch 2/10
183/183 [==============================] - 5s 29ms/step - loss: 0.1488 - accuracy: 0.9392 - val_loss: 0.1863 - val_accuracy: 0.9207
Epoch 3/10
183/183 [==============================] - 3s 15ms/step - loss: 0.1017 - accuracy: 0.9623 - val_loss: 0.1961 - val_accuracy: 0.9222
Epoch 4/10
183/183 [==============================] - 3s 14ms/step - loss: 0.0732 - accuracy: 0.9744 - val_loss: 0.2120 - val_accuracy: 0.9222
Epoch 5/10
183/183 [==============================] - 2s 10ms/step - loss: 0.0503 - accuracy: 0.9834 - val_loss: 0.2566 - val_accuracy: 0.9227
Epoch 6/10
183/183 [==============================] - 2s 10ms/step - loss: 0.0330 - accuracy: 0.9898 - val_loss: 0.2939 - val_accuracy: 0.9180
Epoch 7/10
183/183 [==============================] - 3s 16ms/step - loss: 0.0326 - accuracy: 0.9895 - val_loss: 0.3024 - val_accuracy: 0.919

In [143]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
y_pred[:5]

172/172 [==============================] - 1s 3ms/step


array([[0],
       [0],
       [1],
       [1],
       [0]])

In [144]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

confusion_matrix(y_test, y_pred)

array([[2796,  267],
       [ 221, 2201]])

In [145]:
accuracy_score(y_test, y_pred)

0.9110300820419326

In [146]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.93      0.91      0.92      3063\n           1       0.89      0.91      0.90      2422\n\n    accuracy                           0.91      5485\n   macro avg       0.91      0.91      0.91      5485\nweighted avg       0.91      0.91      0.91      5485\n'